# ETL

In [3]:
#Importamos los paquete necesarios:
import os
import pandas as pd
from sqlalchemy import create_engine

In [38]:
def extractANDclean_CSV(file_csv):
    '''Transformamos un CSV en un datafram y lo renombramos como documento procesado
            IN: archivo CSV
            OUT: DataFrame, csv_renamed
    '''
    print(f"Processing {file_csv} as data")

def extractANDclean_JSON(file_json):
        '''Transformamos un JSON en un datafram y lo renombramos como documento procesado
            IN: archivo JSON
            OUT: DataFrame, JSON_renamed
    ''' 
        
def extractANDclean_TXT(file_txt):
    '''Transformamos un TXT en un datafram y lo renombramos como documento procesado
            IN: archivo TXT
            OUT: DataFrame, TXT_renamed
    '''

def extractANDclean_XLSX(file_xlsx):
    '''Transformamos un XLSX en un datafram y lo renombramos como documento procesado
            IN: archivo XLSX
            OUT: DataFrame, XLSX_renamed
    ''' 

In [39]:
#Damos la ruta al directorio de datos "sucios":
dirty_data = 'dirty_data/'

#Diferenciamos por tipo de documento para poderdirigirlo a la función adecuada:
for doc in os.listdir(dirty_data):
    
    if doc[-4:] == ".csv": extractANDclean_CSV(doc) 
    elif doc[-5:] == ".json": extractANDclean_JSON(doc)
    elif doc[-5:] == ".xlsx": extractANDclean_XLSX(doc)
        #Presuponemos que el separador para los TXT será "|":
    elif doc[-4:] == ".txt": extractANDclean_TXT(doc)
    
    #Levantamos un error si el formato no existe:
    else: 
        raise TypeError(f"Documento con formato no contemplado: {doc}")

Processing dataset_ventas.csv as data
